In [51]:
import json
import re
import datetime

import ckanapi
import geopandas as gpd
import nltk
import numpy as np
import pandas as pd
import petk
import requests

from nltk.corpus import wordnet
from shapely.geometry import mapping, shape, Point, LineString, Polygon
from sklearn.preprocessing import MinMaxScaler

from datetime import datetime as dt
from time import sleep

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/ytan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
PACKAGE_FRAMEWORK = 'catalogue-quality-scores'

RESOURCE_METHODS = 'scoring-methods'
RESOURCE_SCORES = 'catalogue-scorecard'

MODEL_VERSION = 'v0.0.2'

DIMENSIONS = ['interpretability', 'usability', 'metadata', 'freshness', 'granularity', 'completeness', 'accessibility'] # Ordered by importance
METRICS = {
    'completeness': ['completeness'],
    'metadata': ['column_descriptions', 'dataset_metadata'],
    'usability': ['meaningful_names', 'non_constant_columns']
}

DATA_CKAN = {
    'address': 'https://ckanadmin0.intra.prod-toronto.ca',
#     'apikey': ''
}

STORAGE_CKAN = {
    'address': 'https://ckanadmin0.intra.dev-toronto.ca',
    'apikey': '784f11cc-b170-4377-83a3-38ba28662b16'
}

In [7]:
def get_model(ckan, pid=PACKAGE_FRAMEWORK):
    try:
        model = ckan.action.package_show(id=pid)
    except ckanapi.NotAuthorized:
        raise Exception('Permission required to search for the framework package')
    except ckanapi.NotFound:
        raise Exception('Framework package not found')
    
    return {
        r['name']: r for r in model.pop('resources')
    }

In [8]:
def read_datastore(ckan, rid, rows=10000):
    records = []
    
    is_geospatial = False
    
    has_more = True
    while has_more:
        result = ckan.action.datastore_search(id=rid, limit=rows, offset=len(records))
        
        records += result['records']
        has_more = len(records) < result['total']
    
    df = pd.DataFrame(records).drop('_id', axis=1)
    
    if 'geometry' in df.columns:
        df['geometry'] = df['geometry'].apply(lambda x: shape(json.loads(x)))
        
        df = gpd.GeoDataFrame(df, crs={'init': 'epsg:4326'})
    
    return df, [x for x in result['fields'] if x['id'] != '_id']

In [9]:
def load_data(ckan):
    packages = ckan.action.current_package_list_with_resources(limit=500)

    results = []
    for p in packages:
        metadata_fields = ['collection_method', 'limitations', 'civic_issues', 'topics', 'owner_division', 'owner_email']
        metadata_count = sum([1 if f in p and p[f] else 0 for f in metadata_fields])

        for r in p['resources']:
            if not 'datastore_active' in r or not r['datastore_active']:
                continue

            data, fields = read_datastore(ckan, r['id'])

            report = petk.DataReport(data)

            intro = report.introduce(as_dict=True)
            descr = report.describe(as_dict=True)

            fields_described, fields_meaningful = 0, 0
            for f in fields:
                if 'info' in f and f['info']['notes']:
                    fields_described += 1

                eng_words = [wordnet.synsets(x) for x in re.split('\s|_|-', f['id'])]
                if (len([x for x in eng_words if len(x)]) / len(eng_words)) > 0.8:
                    fields_meaningful += 1

            results.append({
                'package': p['name'],
                'resource': r['name'],
                'metrics': {
                    'dataset_metadata': metadata_count/len(metadata_fields),
                    'column_descriptions': 1 - (fields_described / len(fields)),
                    'completeness': 1 - (intro['observations']['missing'] / intro['observations']['total']),
                    'meaningful_names': 1 - (fields_meaningful / len(fields)),
                    'non_constant_columns': 1 - (len([x for x in descr['distinct_count'].values() if x == 1]) / len(descr['distinct_count'].values()))
                }
            })
    
    return results

In [10]:
def calculate_weights(dimensions, data, method='sr'):
    for i, row in enumerate(data):
        row['dimensions'] = {}
        for dim, fields in METRICS.items():
            scores = [ row['metrics'][x] for x in fields ]

            row[dim] = sum(scores)/len(scores)

    N = len(dimensions)
    
    if method == 'sr':
        denom = np.array([ ((1 / (i + 1)) + ((N + 1 - (i + 1)) / N)) for i, x in enumerate(dimensions) ]).sum()
        weights = [ ((1 / (i + 1)) + ((N + 1 - (i + 1)) / N)) / denom for i, x in enumerate(dimensions) ]
    elif method == 'rs':
        denom = np.array([ (N + 1 - (i + 1)) for i, x in enumerate(dimensions)]).sum()
        weights = [ (N + 1 - (i + 1)) / denom for i, x in enumerate(dimensions) ]
    elif method == 'rr':
        denom = np.array([ 1 / (i + 1) for i, x in enumerate(dimensions) ]).sum()
        weights = [ (1 / (i + 1)) / denom for i, x in enumerate(dimensions) ]
    elif method == 're':
        exp = 0.2
        denom = np.array([ (N + 1 - (i + 1)) ** exp for i, x in enumerate(dimensions) ]).sum()
        weights = [ (N + 1 - (i + 1)) ** exp / denom for i, x in enumerate(dimensions) ]
    else:
        raise Exception('Invalid weighting method provided')
    
    return weights

In [11]:
def update_model(ckan, model, storage):
    if not RESOURCE_METHODS in storage:
        r = requests.post(
            '{0}/api/3/action/resource_create'.format(ckan.address),
            data={
                'package_id': PACKAGE_FRAMEWORK,
                'name': RESOURCE_METHODS,
                'format': 'json',
                'is_preview': False
            },
            headers={
                'Authorization': ckan.apikey
            },
            files={
                'upload': ('{0}.json'.format(RESOURCE_METHODS), json.dumps({}))
            }
        )

        storage[RESOURCE_METHODS] = json.loads(r.content)['result']

    r = requests.get(
        storage[RESOURCE_METHODS]['url'],
        headers={
            'Authorization': ckan.apikey
        }
    )

    scoring_methods = json.loads(r.content)
    scoring_methods[MODEL_VERSION] = model

    r = requests.post(
        '{0}/api/3/action/resource_patch'.format(ckan.address),
        data={
            'id': storage[RESOURCE_METHODS]['id']
        },
        headers={
            'Authorization': ckan.apikey
        },
        files={
            'upload': ('{0}.json'.format(RESOURCE_METHODS), json.dumps(scoring_methods))
        }
    )

In [26]:
def update_score(ckan, data, weight, dimensions, storage):
    df = pd.DataFrame(data).set_index(['package', 'resource'])

    scores = pd.DataFrame([weight] * len(df.index))
    scores.index = df.index
    scores.columns = dimensions

    scores = df.multiply(scores)

    df['score'] = scores.sum(axis=1)
    df['score_norm'] = MinMaxScaler().fit_transform(df[['score']])

    df = df.groupby('package').mean()

    df['grade'] = pd.cut(df['score'], bins=[-1, .3, .5, .8, 1], labels=['D','C','B','A'])
    df['grade_norm'] = pd.cut(df['score_norm'], bins=[-1, .3, .5, .8, 1], labels=['D','C','B','A'])

    df['recorded_at'] = dt.now()
    df['model'] = MODEL_VERSION

    df = df.reset_index()

    report = petk.DataReport(df).describe(as_dict=True)

    ckan_fields = {
        'STRING': 'text',
        'NUMERIC': 'float',
        'DATE': 'timestamp'
    }

    fields = [
        {
            'id': k,
            'type': ckan_fields[v]
        } for k, v in report['content_type'].items()
    ]

    if not RESOURCE_SCORES in storage:
        storage[RESOURCE_SCORES] = ckan.action.datastore_create(
            resource={
                'package_id': PACKAGE_FRAMEWORK,
                'name': RESOURCE_SCORES,
                'format': 'csv',
                'is_preview': False
            },
            fields=fields,
            records=[]
        )

    df['recorded_at'] = df['recorded_at'].apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%SZ'))

    ckan.action.datastore_upsert(
        method='insert',
        resource_id=storage[RESOURCE_SCORES]['resource_id'],
        records=df.to_dict(orient='row')
    )

In [18]:
source = ckanapi.RemoteCKAN(**DATA_CKAN)
ckan = ckanapi.RemoteCKAN(**STORAGE_CKAN)

In [19]:
storage = get_model(ckan)

In [20]:
data = load_data(source)

/Users/ytan/miniconda3/envs/open-data/lib/python3.7/site-packages/petk/tools.py:63: RuntimeWarning: invalid value encountered in double_scalars
  'cv': series.std() / series.mean(),


In [21]:
dimensions = [ x for x in DIMENSIONS if x in METRICS ]
weights = calculate_weights(dimensions, data)

In [27]:
model = {
    'aggregation_methods': {
        'metrics_to_dimension': 'avg',
        'dimensions_to_score': 'sum_and_reciprocal'
    },
    'dimensions': [
        {
            'name': dim,
            'rank': i + 1,
            'weight': wgt,
            'metrics': METRICS[dim]
        } for i, (dim, wgt) in enumerate(zip(dimensions, weights))
    ]
}

update_model(ckan, model, storage)
update_score(ckan, data, weights, dimensions, storage)

In [88]:
records = []

is_geospatial = False

has_more = True
while has_more:
    result = source.action.datastore_search(id='b9214fd7-60d1-45f3-8463-a6bd9828f8bf', limit=5000, offset=len(records))

    records += result['records']
    has_more = len(records) < result['total']

df = pd.DataFrame(records).drop('_id', axis=1)

if 'geometry' in df.columns:
    df['geometry'] = df['geometry'].apply(lambda x: shape(json.loads(x)))

    df = gpd.GeoDataFrame(df, crs={'init': 'epsg:4326'})

In [68]:
test = source.action.datastore_search(id='b9214fd7-60d1-45f3-8463-a6bd9828f8bf', limit=100000)['records']
geometry=[Point(json.loads(x['geometry'])['coordinates']) for x in test]
test = gpd.GeoDataFrame(test, crs={'init': 'epsg:4326'}, geometry=geometry).drop(['_id', 'X', 'Y'], axis=1)
test.head()

,ADDRESSNUMBERTEXT,ADDRESSSTREET,ASSETTYPE,BARCODE,BIA,DIRECTION,FROMSTREET,FRONTINGSTREET,ID,LATITUDE,...,OBJECTID,POSTERBOARD,POSTERBOARDSTATUS,SDE_STATE_ID,SIDE,SITEID,STATUS,STYLE,WARD,geometry
0,21,Redway Rd,A,None,None,South,Overlea Blvd,Millwood Rd,SH-02236,43.701603,...,1,No,None,0,West,T2927,Existing,Enseicom,15,POINT (-79.35256628671004 43.70160264566507)
1,2500,Sheppard Ave E,C,None,None,North,Sheppard Ave E,Victoria Park Ave,SH-01079,43.775535,...,2,No,None,0,West,T7426,Existing,Enseicom,17,POINT (-79.32335029056694 43.77553539676025)
2,155,Culford Rd,A,None,None,East,Culford Rd,Lawrence Ave W,SH-01713,43.706311,...,4,No,None,0,South,T5356,Existing,Enseicom,05,POINT (-79.48732379138143 43.70631081680406)
3,5359,Dundas St W,B,None,None,West,Poplar Ave,Dundas St W,SH-00626,43.635766,...,7,Yes,Existing,0,South,T2180,Existing,Enseicom,03,POINT (-79.54014791139981 43.63576633196766)
4,3077,Weston Rd,C,None,Emery Village,North,Mainshep Rd,Weston Rd,SH-02437,43.731806,...,8,No,None,0,West,T11717,Existing,Enseicom,07,POINT (-79.53779539502695 43.73180611693692)


In [90]:
test_report = petk.DataReport(df)

In [91]:
test_report.introduce(as_dict=True)

{'basic': {'memory_usage': 5922457, 'rows': 5684, 'columns': 23},
 'observations': {'total': 130732, 'missing': 12766},
 'columns': {'string': 16, 'numeric': 6, 'geometry': 1},
 'geospatial': {'crs': 'epsg:4326',
  'centroid_location': '-79.39585654203123, 43.72214759653589',
  'bounds': array([-79.63465417,  43.59328834, -79.12309522,  43.83705657]),
  '3d_shapes': 0,
  'points': 5684}}

In [92]:
test_geo = test_report.describe(as_dict=True)
test_geo.keys()

dict_keys(['content_type', 'memory_usage', 'count', 'p_null', 'n_null', 'distinct_count', 'is_constant', 'is_unique', 'p_unique', 'min', 'max', '5%', '25%', '50%', '75%', '95%', 'mean', 'std', 'variance', 'iqr', 'kurtosis', 'skewness', 'sum', 'mad', 'cv', 'p_infinite', 'n_infinite', 'n_zeros', 'p_zeros'])

In [85]:
data

[{'package': 'bodysafe',
  'resource': 'BodySafe Data',
  'metrics': {'dataset_metadata': 0.5,
   'column_descriptions': 0.040000000000000036,
   'completeness': 0.6892552324001087,
   'meaningful_names': 0.84,
   'non_constant_columns': 1.0},
  'dimensions': {},
  'completeness': 0.6892552324001087,
  'metadata': 0.27,
  'usability': 0.9199999999999999},
 {'package': 'street-furniture-transit-shelter',
  'resource': 'Transit shelter data',
  'metrics': {'dataset_metadata': 0.5,
   'column_descriptions': 1.0,
   'completeness': 0.9023498454854205,
   'meaningful_names': 0.4782608695652174,
   'non_constant_columns': 0.9565217391304348},
  'dimensions': {},
  'completeness': 0.9023498454854205,
  'metadata': 0.75,
  'usability': 0.7173913043478262},
 {'package': 'street-furniture-publication-structure',
  'resource': 'Publication structure data',
  'metrics': {'dataset_metadata': 0.5,
   'column_descriptions': 1.0,
   'completeness': 0.9574085365853658,
   'meaningful_names': 0.5,
   'n